In [0]:
import numpy as np 
import pandas as pd 
import dask.dataframe as dd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)

import sys
if not sys.warnoptions:
  import warnings
  warnings.simplefilter('ignore')

This code assumes your data is stored on Google Drive. If you need to import the data in a different way, then you should put that here.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## Functions to reduce the memory usage of a dataframe

In [0]:
def reduce_mem_usage(df, verbose=True):
  """
  Iterate through all the columns of a dataframe and modify the data types
  to reduce memory usage.        
  """
  df = df.copy()
  
  start_mem = df.memory_usage().sum() / 1024**2
  start_mem_gb = start_mem / 1024
  
  numeric_dtype = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  
  for col in df:
    col_type = str(df[col].dtypes)
        
    if col_type == 'object':  # column is an object
      df[col] = df[col].astype('category')

    elif col_type in numeric_dtype:  # column is numerical
      c_min = df[col].min()
      c_max = df[col].max()
      if col_type[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)  
      else:  # column is not int
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
    

  end_mem = df.memory_usage().sum() / 1024**2
  end_mem_gb = end_mem / 1024
  
  if verbose:
    print(f'Memory usage of dataframe is {start_mem:.2f} MB',
        f'/ {start_mem_gb:.2f} GB')
    print(f'Memory usage after optimization is: {end_mem:.2f} MB',
        f'/ {end_mem_gb:.2f} GB')
    mem_dec = 100 * (start_mem - end_mem) / start_mem
    print(f'Decreased by {mem_dec:.1f}%')
    
  return df


def import_data(file, dtypes=None):
  """
  Create a dataframe using dask for faster speed
  """

  ddf = dd.read_csv(file, dtype=dtypes)
  df = ddf.compute()
  return df

## Find all the datatypes.

Since the data is very large, we can reduce the load on the RAM by converting the datatypes in order to use less RAM wherever possible by converting objects to categories and converting 64-bit numerics to 32-bit or 16-bit wherever possible.

Numpy does not have 1-bit integer storage for Boolean, so they are stored as type `int8`. NaNs cannot be stored as integers, so those Boolean columns with NaN values are converted to type `float16`.

In [0]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
}

In [0]:
target = 'HasDetections'
data_id = 'MachineIdentifier'
file_path = 'gdrive/My Drive/build-unit-2-data/'

In [6]:
%time train = import_data(file_path + 'train.csv', dtypes=dtypes)

CPU times: user 4min 29s, sys: 13.8 s, total: 4min 43s
Wall time: 2min 42s


In [7]:
col_stats = []
for col in train.columns:
  # Number of unique values in the column
  unique_vals = train[col].nunique()
  
  # Percent of NaNs in the column
  nan_pct = train[col].isnull().sum() * 100 / train.shape[0]

  # Percent of data that the most common column covers
  col_counts = train[col].value_counts(normalize=True, dropna=False)
  largest_pct = col_counts.values[0] * 100

  # Datatype of the column
  col_type = train[col].dtype

  col_stats.append((col, unique_vals, nan_pct, largest_pct, col_type))

df_header = ['Feature', 'Unique_values', '% of Missing Values',
             'Largest Category %', 'Type']
stats_df = pd.DataFrame(col_stats, columns=df_header)
stats_df.sort_values(['% of Missing Values', 'Largest Category %'],
                     ascending=False)

,Feature,Unique_values,% of Missing Values,Largest Category %,Type
28,PuaMode,2,99.974119,99.974119,category
41,Census_ProcessorClass,3,99.589407,99.589407,category
8,DefaultBrowsersIdentifier,2017,95.141637,95.141637,float32
68,Census_IsFlightingInternal,2,83.044030,83.044030,float16
52,Census_InternalBatteryType,78,71.046809,71.046809,category
71,Census_ThresholdOptIn,2,63.524472,63.524472,float16
75,Census_IsWIMBootEnabled,2,63.439038,63.439038,float16
31,SmartScreen,21,35.610795,48.379658,category
15,OrganizationIdentifier,49,30.841487,47.037662,float16
29,SMode,2,6.027686,93.928812,float16


We want to keep only the relevant features, so we will drop columns with high number of NaNs or highly skewed data.

In [8]:
# Keep only features with less than 50% NaNs and less than 95% dominant category
condition = ((stats_df['% of Missing Values'] <= 50) &
             (stats_df['Largest Category %'] <= 95))
stats_df[condition]

,Feature,Unique_values,% of Missing Values,Largest Category %,Type
0,MachineIdentifier,8921483,0.000000,0.000011,category
2,EngineVersion,70,0.000000,43.098967,category
3,AppVersion,110,0.000000,57.605042,category
4,AvSigVersion,8531,0.000000,1.146861,category
9,AVProductStatesIdentifier,28970,0.405998,65.286960,float32
10,AVProductsInstalled,8,0.405998,69.594853,float16
13,CountryIdentifier,222,0.000000,4.451861,int16
14,CityIdentifier,107366,3.647477,3.647477,float32
15,OrganizationIdentifier,49,30.841487,47.037662,float16
16,GeoNameIdentifier,292,0.002387,17.171237,float16


In [0]:
stats_df_filtered = stats_df[condition]
# Take the relevant features from the 'Feature' column
relevant_features = stats_df_filtered['Feature'].tolist()

In [10]:
relevant_features

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Processor',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'SMode',
 'IeVerIdentifier',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_InternalBat

In [11]:
train.shape

(8921483, 83)

In [12]:
train = train[relevant_features]
print(train.shape)

(8921483, 60)


In [13]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 1639.21 MB / 1.60 GB
Memory usage after optimization is: 1452.03 MB / 1.42 GB
Decreased by 11.4%


In [14]:
%%time
test = import_data(file_path + 'test.csv', dtypes=dtypes)
test[target] = np.nan
print(test.shape)

(7853253, 83)
CPU times: user 4min 11s, sys: 10.1 s, total: 4min 21s
Wall time: 2min 9s


In [15]:
test = test[relevant_features]
print(test.shape)

(7853253, 60)


In [16]:
test = reduce_mem_usage(test)

Memory usage of dataframe is 1533.74 MB / 1.50 GB
Memory usage after optimization is: 1368.97 MB / 1.34 GB
Decreased by 10.7%


In [0]:
num_features = ['Census_SystemVolumeTotalCapacity']

bin_features = ['Census_HasOpticalDiskDrive',
                'Census_IsAlwaysOnAlwaysConnectedCapable',
                'Census_IsSecureBootEnabled',
                'Census_IsTouchEnabled',
                'IsProtected',
                'SMode',
                'Wdft_IsGamer']

noncat_features = num_features + bin_features
# All the other features are categorical
cat_features = [i for i in relevant_features if i not in noncat_features]

In [18]:
train.head()

,MachineIdentifier,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Processor,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,SMode,IeVerIdentifier,SmartScreen,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,1.1.15100.1,4.18.1807.18075,1.273.1735.0,53447.0,1.0,29,128035.0,18.0,35.0,171,x64,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,NaN,Desktop,2668.0,9124.0,4.0,5.0,2340.0,476940.0,HDD,299451.0,0,4096.0,Desktop,18.906250,1440.0,900.0,Desktop,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,IS_GENUINE,Retail,Retail,628.0,36144.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,1.1.14600.4,4.13.17134.1,1.263.48.0,53447.0,1.0,93,1482.0,18.0,119.0,64,x64,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,NaN,Notebook,2668.0,91656.0,4.0,5.0,2404.0,476940.0,HDD,102385.0,0,4096.0,Notebook,13.898438,1366.0,768.0,Mobile,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,OFFLINE,Retail,NOT_SET,628.0,57858.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,1.1.15100.1,4.18.1807.18075,1.273.1341.0,53447.0,1.0,86,153579.0,18.0,64.0,49,x64,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0.0,137.0,RequireAdmin,Desktop,4908.0,317701.0,4.0,5.0,1972.0,114473.0,SSD,113907.0,0,4096.0,Desktop,21.500000,1920.0,1080.0,Desktop,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,IS_GENUINE,OEM:NONSLP,Retail,142.0,52682.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,1.1.15100.1,4.18.1807.18075,1.273.1527.0,53447.0,1.0,88,20710.0,NaN,117.0,115,x64,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,ExistsNotSet,Desktop,1443.0,275890.0,4.0,5.0,2272.0,238475.0,UNKNOWN,227116.0,0,4096.0,MiniTower,18.500000,1366.0,768.0,Desktop,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,IS_GENUINE,OEM:NONSLP,Retail,355.0,20050.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,1.1.15100.1,4.18.1807.18075,1.273.1379.0,53447.0,1.0,18,37376.0,NaN,277.0,75,x64,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0.0,137.0,RequireAdmin,Notebook,1443.0,331929.0,4.0,5.0,2500.0,476940.0,HDD,101900.0,0,6144.0,Portable,14.000000,1366.0,768.0,Mobile,0.000000e+00,10.0.17134.191,amd64,rs4_release,17134,191,Core,CORE,Update,8.0,31,FullAuto,IS_GENUINE,Retail,Retail,355.0,19844.0,0,0,0.0,0.0,1.0,1


In [19]:
test.head()

,MachineIdentifier,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Processor,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,SMode,IeVerIdentifier,SmartScreen,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,1.1.15400.5,4.18.1810.5,1.281.501.0,53447.0,1.0,43,58552.0,18.0,53.0,42,x64,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,NaN,108.0,NaN,Notebook,2688.0,30661.0,4.0,5.0,3064.0,488386.0,SSD,123179.0,0,8192.0,Notebook,15.500000,1920.0,1080.0,Mobile,8.0,10.0.15063.1387,amd64,rs2_release,15063,1387,Core,CORE,Reset,37.0,158,AutoInstallAndRebootAtMaintenanceTime,IS_GENUINE,OEM:DM,Retail,807.0,8554.0,1,0,0.0,0.0,7.0,NaN
1,00000176ac758d54827acd545b6315a5,1.1.15400.4,4.18.1809.2,1.279.301.0,53447.0,1.0,68,71395.0,NaN,276.0,74,x64,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.180502-1908,Home,1.0,NaN,117.0,RequireAdmin,Notebook,2206.0,242491.0,4.0,5.0,3410.0,1907729.0,HDD,1882352.0,0,8192.0,Notebook,15.500000,1366.0,768.0,Mobile,0.0,10.0.16299.611,amd64,rs3_release_svc_escrow,16299,611,Core,CORE,UUPUpgrade,7.0,30,AutoInstallAndRebootAtMaintenanceTime,IS_GENUINE,Retail,Retail,554.0,33105.0,1,0,0.0,1.0,12.0,NaN
2,0000019dcefc128c2d4387c1273dae1d,1.1.15300.6,4.18.1809.2,1.277.230.0,49480.0,2.0,201,66202.0,NaN,267.0,251,x64,14393,768,rs1,14393.2189.amd64fre.rs1_release.180329-1711,Home,1.0,NaN,98.0,RequireAdmin,Notebook,585.0,189533.0,2.0,5.0,2096.0,29820.0,SSD,28678.0,0,4096.0,Notebook,13.898438,1280.0,720.0,Mobile,0.0,10.0.14393.2189,amd64,rs1_release,14393,2189,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,35.0,148,UNKNOWN,IS_GENUINE,OEM:DM,Retail,556.0,63396.0,1,0,0.0,1.0,11.0,NaN
3,0000055553dc51b1295785415f1a224d,1.1.15400.5,4.18.1810.5,1.281.664.0,42160.0,2.0,29,120917.0,NaN,35.0,171,x64,16299,768,rs3,16299.15.amd64fre.rs3_release.170928-1534,Home,1.0,NaN,117.0,RequireAdmin,Notebook,2668.0,171228.0,2.0,5.0,1985.0,476940.0,HDD,439345.0,0,4096.0,Notebook,14.000000,1366.0,768.0,Mobile,0.0,10.0.16299.371,amd64,rs3_release,16299,371,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,26.0,119,Notify,IS_GENUINE,OEM:DM,Retail,628.0,26320.0,1,0,0.0,0.0,10.0,NaN
4,00000574cefffeca83ec8adf9285b2bf,1.1.15400.4,4.18.1809.2,1.279.236.0,53447.0,1.0,171,124736.0,18.0,211.0,182,x64,16299,768,rs3,16299.15.amd64fre.rs3_release.170928-1534,Home,1.0,NaN,117.0,RequireAdmin,Notebook,585.0,189538.0,4.0,5.0,3394.0,476940.0,HDD,461506.0,1,2048.0,Notebook,15.500000,1366.0,768.0,Mobile,0.0,10.0.16299.371,amd64,rs3_release,16299,371,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,29.0,125,UNKNOWN,IS_GENUINE,Retail,Retail,556.0,63269.0,1,0,0.0,1.0,3.0,NaN


#### Target Feature

The target feature is almost perfectly balanced, very close to 50-50.

In [20]:
train['HasDetections'].value_counts(normalize=True)

0    0.500207
1    0.499793
Name: HasDetections, dtype: float64

## Cleaning the Train and Test Datasets

In [0]:
freq_encoding_list = []
nan_minus1 = []
nan_0 = []
nan_1 = []

In [0]:


def impute_features(X):
  """
  Custom imputation for numerical, binary, and categorical features.
  """
  X = X.copy()
  
  # Numerical Features
  # Replace NaN values with -1
  for feature in num_features:
    X[feature] = X[feature].fillna(-1)
    X[feature] = X[feature].astype('int32')
    nan_minus1.append(feature)
  
  # Binary Features
  # Replace NaN values with the column's mode
  for feature in bin_features:
    X[feature] = X[feature].fillna(X[feature].mode()[0])
    X[feature] = X[feature].astype('int8')
    if X[feature].mode()[0] == 0:
      nan_0.append(feature)
    else:
      nan_1.append(feature)
  
  # Categorical Features
  # Replace all "numerical" categorical feature NaNs with -1
  # We need to handle all the other features by hand
  features_by_hand = []
  for feature in cat_features:
    categorical = pd.api.types.is_categorical_dtype(X[feature])
    if not categorical:
      if feature != target:
        # Replace the NaNs with -1
        X[feature] = X[feature].fillna(-1)
        X[feature] = X[feature].astype('int32')
        nan_minus1.append(feature)
    else:
      # Add to the list of features to correct by hand
      features_by_hand.append(feature)

  return X, features_by_hand

In [0]:
train, features_by_hand_train = impute_features(train)
test, features_by_hand_test = impute_features(test)

In [24]:
features_by_hand_train

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'Processor',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing']

In [0]:
def wrangle(X, features_by_hand):
  """
  Wrangle the train and test dataframes in the same way.
  In order the handle the other features above, we should:
    1. Convert all values to lowercase.
    2. Merge different spellings of the features together.
    3. Set NaNs to 'unknown'.
    4. Merge or set rare values to 'unknown'.
    5. Label all values with numerics.
    6. Add feature to its corresponding encoding list based on feature amount.
  """
  X = X.copy()

  # Convert all values to lowercase.
  for feature in features_by_hand:
    X[feature] = X[feature].str.lower()

  # Feature: MachineIdentifier
  # Nothing needed

  # Feature: EngineVersion
  freq_encoding_list.append('EngineVersion')

  # Feature: AppVersion
  freq_encoding_list.append('AppVersion')

  # Feature: AvSigVersion
  X['AvSigVersion'] = X['AvSigVersion'].replace('1.2&#x17;3.1144.0', '1.2173.1144.0')
  freq_encoding_list.append('AvSigVersion')

  # Feature: Processor
  # Nothing needed

  # Feature: OsPlatformSubRelease
  # Nothing needed

  # Feature: OsBuildLab
  X['OsBuildLab'] = X['OsBuildLab'].astype('category')
  X['OsBuildLab'] = X['OsBuildLab'].cat.add_categories(['unknown'])
  X['OsBuildLab'] = X['OsBuildLab'].fillna('unknown')
  freq_encoding_list.append('OsBuildLab')

  # Feature: SkuEdition
  X['SkuEdition'] = X['SkuEdition'].astype(str)
  X.loc[X['SkuEdition'] != 'Home', 'SkuEdition'] = 'Pro'
  X['SkuEdition'] = X['SkuEdition'].astype('category')
  X['SkuEdition'] = X['SkuEdition'].cat.remove_unused_categories()

  # Feature: SmartScreen
  X['SmartScreen'] = X['SmartScreen'].astype('category')
  X['SmartScreen'] = X['SmartScreen'].fillna('existsnotset')
  X['SmartScreen'] = X['SmartScreen'].astype(str)
  condition = X['SmartScreen'].isin(['requireadmin',
                                     'existsnotset',
                                     'off',
                                     'warn']) == False
  X.loc[condition, 'SmartScreen'] = 'prompt'
  X['SmartScreen'] = X['SmartScreen'].astype('category')
  X['SmartScreen'] = X['SmartScreen'].cat.remove_unused_categories()

  # Feature: Census_MDC2FormFactor
  X['Census_MDC2FormFactor_new'] = X['Census_MDC2FormFactor']
  features_by_hand.append('Census_MDC2FormFactor_new')
  X['Census_MDC2FormFactor_new'] = X['Census_MDC2FormFactor_new'].astype(str)
  X['Census_MDC2FormFactor_new'] = X['Census_MDC2FormFactor_new'].apply(rename_Census_MDC2FormFactor_new)
  X['Census_MDC2FormFactor_new'] = X['Census_MDC2FormFactor_new'].astype('category')
  X['Census_MDC2FormFactor_new'] = X['Census_MDC2FormFactor_new'].cat.remove_unused_categories()

  # Feature: Census_PrimaryDiskTypeName
  X['Census_PrimaryDiskTypeName'] = X['Census_PrimaryDiskTypeName'].astype('category')
  X['Census_PrimaryDiskTypeName'] = X['Census_PrimaryDiskTypeName'].replace('unspecified', 'unknown')
  X['Census_PrimaryDiskTypeName'] = X['Census_PrimaryDiskTypeName'].fillna('unknown')
  X['Census_PrimaryDiskTypeName'] = X['Census_PrimaryDiskTypeName'].cat.remove_unused_categories()

  # Feature: Census_ChassisTypeName
  X['Census_ChassisTypeName'] = X['Census_ChassisTypeName'].fillna('unknown')
  X['Census_ChassisTypeName'] = X['Census_ChassisTypeName'].astype(str)
  X['Census_ChassisTypeName'] = X['Census_ChassisTypeName'].apply(rename_Census_ChassisTypeName)
  X['Census_ChassisTypeName'] = X['Census_ChassisTypeName'].astype('category')
  X['Census_ChassisTypeName'] = X['Census_ChassisTypeName'].cat.remove_unused_categories()

  # Feature: Census_PowerPlatformRoleName
  X['Census_PowerPlatformRoleName'] = X['Census_PowerPlatformRoleName'].astype('category')
  X['Census_PowerPlatformRoleName'] = X['Census_PowerPlatformRoleName'].replace('unspecified', 'unknown')
  X['Census_PowerPlatformRoleName'] = X['Census_PowerPlatformRoleName'].fillna('unknown')
  X['Census_PowerPlatformRoleName'] = X['Census_PowerPlatformRoleName'].cat.remove_unused_categories()

  # Feature: Census_OSVersion
  freq_encoding_list.append('Census_OSVersion')

  # Feature: Census_OSArchitecture
  # Nothing needed

  # Feature: Census_OSBranch
  # Nothing needed

  # Feature: Census_OSEdition
  X['Census_OSEdition'] = X['Census_OSEdition'].astype('category')
  X['Census_OSEdition'] = X['Census_OSEdition'].cat.add_categories(['unknown'])
  X['Census_OSEdition'] = X['Census_OSEdition'].fillna('unknown')
  X['Census_OSEdition'] = X['Census_OSEdition'].astype(str)
  X['Census_OSEdition'] = X['Census_OSEdition'].apply(rename_Census_OSEdition)
  X['Census_OSEdition'] = X['Census_OSEdition'].astype('category')
  X['Census_OSEdition'] = X['Census_OSEdition'].cat.remove_unused_categories()

  # Feature: Census_OSSkuName
  X['Census_OSSkuName'] = X['Census_OSSkuName'].astype(str)
  X['Census_OSSkuName'] = X['Census_OSSkuName'].apply(rename_Census_OSEdition)
  X['Census_OSSkuName'] = X['Census_OSSkuName'].astype('category')
  X['Census_OSSkuName'] = X['Census_OSSkuName'].cat.remove_unused_categories()

  # Feature: Census_OSInstallTypeName
  # Nothing needed

  # Feature: Census_OSWUAutoUpdateOptionsName
  # Nothing needed

  # Feature: Census_GenuineStateName
  X['Census_GenuineStateName'] = X['Census_GenuineStateName'].astype('category')
  X['Census_GenuineStateName'] = X['Census_GenuineStateName'].replace('tampered', 'unknown')
  X['Census_GenuineStateName'] = X['Census_GenuineStateName'].fillna('unknown')
  X['Census_GenuineStateName'] = X['Census_GenuineStateName'].cat.remove_unused_categories()

  # Feature: Census_ActivationChannel
  X['Census_ActivationChannel'] = X['Census_ActivationChannel'].astype(str)
  X['Census_ActivationChannel'] = X['Census_ActivationChannel'].apply(rename_Census_ActivationChannel)
  X['Census_ActivationChannel'] = X['Census_ActivationChannel'].astype('category')

  # Feature: Census_FlightRing
  X['Census_FlightRing'] = X['Census_FlightRing'].astype('category')
  X['Census_FlightRing'] = X['Census_FlightRing'].replace('disabled', 'not_set')
  X['Census_FlightRing'] = X['Census_FlightRing'].replace(['osg', 'canary', 'invalid'], 'unknown')
  X['Census_FlightRing'] = X['Census_FlightRing'].fillna('unknown')
  X['Census_FlightRing'] = X['Census_FlightRing'].cat.remove_unused_categories()

  return X, freq_encoding_list

def rename_Census_MDC2FormFactor_new(x):
  x = x.lower()
  if 'server' in x:
    return 'server'
  elif 'tablet' in x:
    return 'tablet'                  
  else:
    return x

def rename_Census_ChassisTypeName(x):
  x = x.lower()
  if 'laptop' in x:
    return 'notebook'
  elif 'other' in x:
    return 'unknown'                  
  else:
    return x

def rename_Census_OSEdition(x):
  x = x.lower()
  if 'core' in x:
    return 'core'
  elif 'pro' in x:
    return 'pro'
  elif 'enterprise' in x:
    return 'enterprise'
  elif 'server' in x:
    return 'server'
  elif 'home' in x:
    return 'home'
  elif 'education' in x:
    return 'education'
  elif 'cloud' in x:
    return 'cloud'
  else:
    return x

def rename_Census_ActivationChannel(x):
  x = x.lower()
  if 'oem' in x:
    return 'oem'
  elif 'volume' in x:
    return 'volume'
  elif 'retail' in x:
    return 'retail'
  else:
    return x

In [26]:
%time train, freq_encoding_list = wrangle(train, features_by_hand_train)
%time test, freq_encoding_list = wrangle(test, features_by_hand_test)

CPU times: user 1min, sys: 7.86 s, total: 1min 8s
Wall time: 1min 8s
CPU times: user 53.7 s, sys: 1.71 s, total: 55.4 s
Wall time: 55.3 s


In [28]:
freq_encoding_list

['EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'OsBuildLab',
 'Census_OSVersion',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'OsBuildLab',
 'Census_OSVersion']

In [29]:
features_by_hand_train

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'Processor',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing',
 'Census_MDC2FormFactor_new']

In [30]:
nan_minus1

['Census_SystemVolumeTotalCapacity',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'IeVerIdentifier',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Wdft_RegionIdentifier',
 'Census_SystemVolumeTotalCapacity',
 'AVProductStatesIdentifier',
 'AVProductsIn

In [31]:
nan_0

['Census_HasOpticalDiskDrive',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Census_IsSecureBootEnabled',
 'Census_IsTouchEnabled',
 'SMode',
 'Wdft_IsGamer',
 'Census_HasOpticalDiskDrive',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Census_IsSecureBootEnabled',
 'Census_IsTouchEnabled',
 'SMode',
 'Wdft_IsGamer']

In [32]:
nan_1

['IsProtected', 'IsProtected']

In [35]:
%time train.to_csv(file_path + 'train_clean.csv', index=False)
%time test.to_csv(file_path + 'test_clean.csv', index=False)

CPU times: user 6min 16s, sys: 4.82 s, total: 6min 21s
Wall time: 7min 31s
CPU times: user 5min 38s, sys: 4.71 s, total: 5min 42s
Wall time: 6min 57s
